# imports

In [1]:
import pandas as pd

# Preprocess dataframe

## Load data and concat two years

In [2]:
data_2018 = pd.read_csv("data2018.csv", encoding="cp1251")
data_2019 = pd.read_csv("data2019.csv", encoding="cp1251")
df = pd.concat([data_2018, data_2019])
df.head()

,Рік,Область,Учбовий заклад,Спеціальність,Факультет,Місце у рейтингу,Пріоритет,Конкурсний бал,Статус заяви,Інфо про спеціальність,Квоти
0,2018,м. Київ,Вищий навчальний заклад «Відкритий міжнародний...,016 Спеціальна освіта,-,1,—,127.5,До наказу (контракт),Українська мова та література (ЗНО): 129\r\nБі...,—
1,2018,м. Київ,Вищий навчальний заклад «Відкритий міжнародний...,016 Спеціальна освіта,-,2,—,160.9,Рекомендовано (контракт),Українська мова та література (ЗНО): 158\r\nБі...,—
2,2018,м. Київ,Вищий навчальний заклад «Відкритий міжнародний...,016 Спеціальна освіта,-,3,—,151.4,Рекомендовано (контракт),Українська мова та література (ЗНО): 152\r\nБі...,—
3,2018,м. Київ,Вищий навчальний заклад «Відкритий міжнародний...,016 Спеціальна освіта,-,4,—,150.6,Рекомендовано (контракт),Українська мова та література (ЗНО): 163\r\nБі...,—
4,2018,м. Київ,Вищий навчальний заклад «Відкритий міжнародний...,016 Спеціальна освіта,-,5,—,147.7,Рекомендовано (контракт),Українська мова та література (ЗНО): 168\r\nБі...,—


## Rename, remove and clean columns
0 - didn't pass at all

1 - pass to contract

2 - pass to budget

In [3]:
df.columns = [
    "year",
    "district",
    "university",
    "specialty",
    "faculty",
    "rating",
    "priority",
    "points",
    "status",
    "info",
    "quotas"
]
df = df.drop(["year", "university", "faculty", "info", "quotas", "rating", "priority"], axis=1)
df["specialty"] = df["specialty"].map(lambda val: val[:3])
df["status"] = df["status"].map({
    'До наказу (контракт)': 1,
    'Рекомендовано (контракт)': 1,
    'Відмова': 0,
    'Скасовано НЗ': 0,
    'Скасовано (втрата пріор.)': 0,
    'Зареєстровано': 1,
    'Затримано': 0,
    'Відраховано': 0,
    'До наказу (бюджет)': 2,
    'Відхилено (бюджет)': 0,
    'Допущено': 1,
    'Рекомендовано (бюджет)': 2,
    'Заява надійшла з сайту': 1,
    'Відхилено (контракт)': 0,
    'Допущено (контракт, за ріш. ПК)': 1,
    'Рекомендовано (к)': 1,
    'До наказу (к)': 1,
    'До наказу (б)': 2,
    'Відхилено (б)': 0,
    'Рекомендовано (б)': 2,
    'Відхилено (к)': 0,
    'Допущено (к, за ріш. ПК)': 1
})
df.status = df.status.dropna()
df.head()

,district,specialty,points,status
0,м. Київ,016,127.5,1
1,м. Київ,016,160.9,1
2,м. Київ,016,151.4,1
3,м. Київ,016,150.6,1
4,м. Київ,016,147.7,1


## Get values for frontend

In [4]:
df.district.unique()

array(['м. Київ', 'Вінницька область', 'Волинська область',
       'Дніпропетровська область', 'Донецька область',
       'Житомирська область', 'Закарпатська область',
       'Запорізька область', 'Івано-Франківська область',
       'Київська область', 'Кіровоградська область', 'Луганська область',
       'Львівська область', 'Миколаївська область', 'Одеська область',
       'Полтавська область', 'Рівненська область', 'Сумська область',
       'Тернопільська область', 'Харківська область',
       'Херсонська область', 'Хмельницька область', 'Черкаська область',
       'Чернівецька область', 'Чернігівська область'], dtype=object)

In [5]:
df.specialty.unique()

array(['016', '022', '029', '035', '053', '061', '071', '072', '073',
       '075', '081', '091', '121', '122', '123', '161', '181', '226',
       '227', '231', '242', '274', '281', '291', '051', '076', '223',
       '292', '015', '054', '124', '125', '273', '275', '141', '151',
       '271', '126', '171', '172', '173', '014', '241', '293', '023',
       '101', '131', '133', '144', '183', '191', '192', '193', '152',
       '206', '021', '027', '031', '032', '033', '034', '052', '102',
       '103', '104', '105', '106', '111', '112', '113', '162', '224',
       '024', '025', '026', '028', '182', '012', '013', '017', '134',
       '142', '153', '163', '186', '263', '272', '011', '041', '232',
       '136', '143', '184', '132', '194', '187', '201', '202', '203',
       '204', '205', '207', '208', '211', '185', '262', '256', '251',
       '145', '261', '135', '229', '255'], dtype=object)

## Split into two datasets

- The first defines people who didn't pass to contract and didn't pass at all as the same people. This model is used to check people who wants to apply for budget
- The second defines people who pass to contract and to budget as the same category, since if you pass to budget you'll pass to contract

In [6]:
df_budget = df.copy()
df_contract = df.copy()
df_budget["status"] = df_budget["status"].map({
    1: 0,
    2: 1,
    0: 0
})
df_contract["status"] = df_contract["status"].map({
    1: 1,
    2: 1,
    0: 0
})


In [7]:
df_budget.head()

,district,specialty,points,status
0,м. Київ,016,127.5,0
1,м. Київ,016,160.9,0
2,м. Київ,016,151.4,0
3,м. Київ,016,150.6,0
4,м. Київ,016,147.7,0


In [8]:
df_contract.head()

,district,specialty,points,status
0,м. Київ,016,127.5,1
1,м. Київ,016,160.9,1
2,м. Київ,016,151.4,1
3,м. Київ,016,150.6,1
4,м. Київ,016,147.7,1


## Dump cleaned_data set

In [9]:
df_budget.to_csv("budget.csv", index=False)
df_contract.to_csv("contract.csv", index=False)